In [1]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

import test
from utils.data_utils import get_loader
from models_swin.ms_swin_transformer import *
from models_swin.swin_transformer import SwinTransformer
import models_swin.ms_backup as ms_b

import matplotlib.pyplot as plt
# from visual import featuremap
import visual
import timm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

args=test.parse_option()

In [ ]:
# data
_, test_loader = get_loader(args)
# train_list = list(enumerate(train_loader))
test_list = list(enumerate(test_loader))

# for i in range(0,3):
img, label = next(iter(test_loader))

plt.imshow(img[0,:].permute(1,2,0).detach().cpu().numpy())
# plt.imshow(test_loader.dataset.test_img[0])

In [4]:
# model
# model_ckpt = torch.load('output/sample_run_swin_t_no_checkpoint.bin')
model_ckpt = torch.load('output/ms_sample_run_swin_t_no_check_checkpoint.bin')

model=MSSwinTransformer(img_size=448, num_classes=200, num_feature_layers=1, detail_features=True)
# model=SwinTransformer(img_size=448, num_classes=200)
# model=ms_b.MSSwinTransformer(img_size=448, num_classes=200)
model.load_state_dict(model_ckpt['model'], strict=False)
model.eval()
# x=torch.randn((16,3,448,448))
x = test_list[0][1][0]
f=model.forward_features(x)
logits=model(x)
# f.shape

In [ ]:
# multi-layer feature map
layers = [2, 2, 6, 2]
lf = model.layer_features
vv=[]
for i in range(len(layers)):
    for j in range(layers[i]):
        stage = i
        block = j
        edge=6
        corr,f = visual.featuremap(lf[stage][block], num_batch=0, edge=edge, start_ch=0)
        s,v,d = np.linalg.svd(corr)
        vv.append(v)
        plt.figure()
        plt.suptitle('stage={0},block={1}'.format(stage, block), fontsize=14)
        plt.subplot(1, 2, 1)
        plt.imshow(corr)    
        plt.title("channel correlation")
        plt.subplot(1, 2, 2)
        plt.imshow(f)
        plt.title("feature map")
        
        
        # plt.colorbar()
#         plt.savefig('./visual/stage={0}_block={1}_edge{2}.png'.format(stage,block,edge),dpi=300)
        plt.show()                                    

In [ ]:
s=3
b=-1
plt.imshow(lf[s][b].mean(2).view(-1,int(112/2**s),int(112/2**s)).detach()[5,:,:])
lf[3][1].shape
plt.colorbar()

In [ ]:
## logits
# s=torch.nn.Sigmoid()(logits)
# sd=s.detach()
# plt.plot(sd[1,:])
plt.plot(logits[2,:].detach())
print(torch.argmax(logits[2,:]))

In [ ]:
tl=iter(test_loader)

In [ ]:
test_model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True)

In [ ]:
# CAM

from pytorch_grad_cam import GradCAM, \
                             ScoreCAM, \
                             GradCAMPlusPlus, \
                             AblationCAM, \
                             XGradCAM, \
                             EigenCAM, \
                             EigenGradCAM

from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
                                         deprocess_image, \
                                         preprocess_image



In [ ]:
plt.imshow(img[0,:].permute(1,2,0).detach().cpu().numpy())

In [ ]:
os.listdir('models_swin/')

In [ ]:
os.path.join('models_swin/', 'ms_backup.py')

In [ ]:
# confusion matrix
test_loader.dataset[32][1]
labels=[test_loader.dataset[i][1] for i in range(len(test_loader.dataset))]

In [ ]:
# confusion matrix
pred = np.load('confusion_base.npy')
conf=np.zeros((200,200))
for n, i in enumerate(labels):
    j = np.argmax(pred[n])
    conf[i][j]+=1
    
plt.figure(dpi=300)
plt.imshow((conf.T/conf.sum(1)).T)
plt.colorbar()
plt.title('confusion matrix(swin_base)')
plt.xlabel('pred')
plt.ylabel('label')
plt.savefig('confusion_matrix_base.png',dpi=300)
plt.show()

In [ ]:
conf.trace()/conf.sum()

In [ ]:
s=3
H = int(112/2**s)
plt.figure()
for b in range(2):
    plt.subplot(2,1,b+1)
    feature=lf[s][b]
    feature=feature.mean(2).squeeze()
    vv=feature.view(8,H,H)
    plt.imshow(vv[7])
    plt.colorbar()

In [ ]:
feature=lf[2][1]
feature=feature.mean(2).squeeze()
vv=feature.view(8,28,28)
plt.imshow(vv[7])
plt.colorbar()

In [ ]:
for i in range(len(vv)):
    plt.figure()
    plt.plot(vv[i])
    print(vv[i][50])

In [5]:
from aff import fusion

In [18]:
a=torch.randn((3,5,5))
print(a)
print(a.prod(0))
print(a.prod(0)**(1/3))

tensor([[[-0.8575,  0.8771, -0.4475, -1.9260, -0.0536],
         [-0.1054, -2.0634, -1.3432, -0.9708, -0.5831],
         [ 0.6183,  0.6156,  1.3839, -1.0138, -1.6934],
         [ 0.6169, -1.2540,  0.6231,  1.3413, -0.0665],
         [ 0.4537,  0.0672,  0.6152,  0.9737,  1.5877]],

        [[ 0.1877, -0.0726,  0.9181, -1.3993, -0.1569],
         [ 1.1534, -0.9384,  0.3260,  0.4129,  0.9172],
         [-0.7366, -0.9122,  0.2868, -0.5730, -0.5904],
         [-1.1469,  0.3358,  0.1498,  0.1514,  0.4370],
         [ 1.3599, -0.5992,  0.1621,  0.3121,  0.6999]],

        [[-0.1285,  0.1215,  0.6114, -0.5164,  0.8864],
         [ 1.5016,  0.0780, -0.7000, -0.3608, -0.9741],
         [-1.4275,  1.6934,  0.1261,  0.1998,  1.8123],
         [-0.5990, -0.8963,  1.7971, -0.3297,  0.7304],
         [-0.4662,  0.3959, -0.8960,  0.3101, -0.6885]]])
tensor([[ 0.0207, -0.0077, -0.2512, -1.3918,  0.0075],
        [-0.1825,  0.1510,  0.3065,  0.1446,  0.5210],
        [ 0.6501, -0.9510,  0.0501,  0.1161,

In [14]:
aff=fusion.AFF

In [19]:
tb=torch.utils.tensorboard

In [20]:
dir(tb)

['FileWriter',
 'RecordWriter',
 'SummaryWriter',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_convert_np',
 '_embedding',
 '_onnx_graph',
 '_proto_graph',
 '_pytorch_graph',
 '_utils',
 'summary',
 'writer']

In [24]:
a=torch.randn((1,3,10,10))
b=torch.nn.AdaptiveAvgPool2d(1)(a)
print(b.shape)

torch.Size([1, 3, 1, 1])


In [31]:
a=torch.nn.Parameter(torch.Tensor(1,2,3))

In [36]:
a=10
if a>3:
    print('>3')
elif a>1:
    print('>1')

>3
